In [1]:
import nfl_data_py as nfl
import pandas as pd
import matplotlib.pyplot as plt
from adjustText import adjust_text
from matplotlib.patches import Rectangle
import numpy as np

from functions import win_loss

In [2]:

week_by_week2024 = nfl.import_weekly_data([2024], downcast=True)
play_by_play2024 = nfl.import_pbp_data([2024], downcast=True, cache=False, alt_path=None)
schedule2024 = nfl.import_schedules([2024])
seasonal2024 = nfl.import_seasonal_data([2024], "REG")
winstotal2024 = nfl.import_win_totals([2024])

Downcasting floats.
2024 done.
Downcasting floats.


In [ ]:
for columns in seasonal2024.columns:
    print(columns)

In [ ]:
# printing the columns of play_byPlay so I don't mess up names lol
for column in play_by_play2024.columns:
    print(column)

In [4]:
for column in schedule2024.columns:
    print(column)
    

game_id
season
game_type
week
gameday
weekday
gametime
away_team
away_score
home_team
home_score
location
result
total
overtime
old_game_id
gsis
nfl_detail_id
pfr
pff
espn
ftn
away_rest
home_rest
away_moneyline
home_moneyline
spread_line
away_spread_odds
home_spread_odds
total_line
under_odds
over_odds
div_game
roof
surface
temp
wind
away_qb_id
home_qb_id
away_qb_name
home_qb_name
away_coach
home_coach
referee
stadium_id
stadium


In [3]:
# printing the columns of week_by_week so I don't mess up names lol
for columns in week_by_week2024.columns: 
    print (columns)


# small_wbw_df = week_by_week2024[["recent_team", "player_name", "fantasy_points_ppr", "wopr", "dakota", "position"]]

# # justin jefferson
# small_wbw_df[(small_wbw_df["recent_team"] == "CIN") & (small_wbw_df["position"] == "WR")]


player_id
player_name
player_display_name
position
position_group
headshot_url
recent_team
season
week
season_type
opponent_team
completions
attempts
passing_yards
passing_tds
interceptions
sacks
sack_yards
sack_fumbles
sack_fumbles_lost
passing_air_yards
passing_yards_after_catch
passing_first_downs
passing_epa
passing_2pt_conversions
pacr
dakota
carries
rushing_yards
rushing_tds
rushing_fumbles
rushing_fumbles_lost
rushing_first_downs
rushing_epa
rushing_2pt_conversions
receptions
targets
receiving_yards
receiving_tds
receiving_fumbles
receiving_fumbles_lost
receiving_air_yards
receiving_yards_after_catch
receiving_first_downs
receiving_epa
receiving_2pt_conversions
racr
target_share
air_yards_share
wopr
special_teams_tds
fantasy_points
fantasy_points_ppr


In [ ]:
def get_team_schedule (schedule_df,team_name): 
    # filter rows where the team is either the home or away team
    team_home_games = schedule_df[schedule_df['home_team'] == team_name]
    team_away_games = schedule_df[schedule_df['away_team'] == team_name]

    # create lists of matchups
    home_games = [row['away_team'] for _, row in team_home_games.iterrows()]
    away_games = [row['home_team'] for _, row in team_away_games.iterrows()]

    # Combine the lists and sort by the 'week' column
    full_schedule = (
        team_home_games[['week']].assign(matchup=home_games)
        .append(team_away_games[['week']].assign(matchup=away_games))
        .sort_values(by='week')
        .reset_index(drop=True)
    )

    # Extract the list of matchups
    schedule_list = full_schedule['matchup'].tolist()
    return schedule_list
    
    
    
    

team_name = "NYJ" 
nyj_schedule = get_team_schedule(schedule2024, team_name)
print(nyj_schedule)

              game_id  season game_type  week     gameday weekday gametime  \
6721   2024_01_NYJ_SF    2024       REG     1  2024-09-09  Monday    20:15   
6731  2024_02_NYJ_TEN    2024       REG     2  2024-09-15  Sunday    13:00   
6771  2024_05_NYJ_MIN    2024       REG     5  2024-10-06  Sunday    09:30   
6810  2024_07_NYJ_PIT    2024       REG     7  2024-10-20  Sunday    20:20   
6820   2024_08_NYJ_NE    2024       REG     8  2024-10-27  Sunday    13:00   
6854  2024_10_NYJ_ARI    2024       REG    10  2024-11-10  Sunday    16:25   
6902  2024_14_NYJ_MIA    2024       REG    14  2024-12-08  Sunday    13:00   
6918  2024_15_NYJ_JAX    2024       REG    15  2024-12-15  Sunday    13:00   
6949  2024_17_NYJ_BUF    2024       REG    17  2024-12-29  Sunday    13:00   

     away_team  away_score home_team  ...  wind  away_qb_id  home_qb_id  \
6721       NYJ        19.0        SF  ...  10.0  00-0023459  00-0037834   
6731       NYJ        24.0       TEN  ...  13.0  00-0023459  00-00391

C:\Users\ieyua\AppData\Local\Temp\ipykernel_14460\2571315519.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(team_away_games[['week']].assign(matchup=away_games))


In [ ]:
smallWeekByWeek_df = week_by_week2024[['position', 'position_group']]

# getting fantasy defense ranking through week by week data
def fantasy_defense_rankings(week_by_week_df): 
    smallFantasy_df = week_by_week_df[['fantasy_points_ppr', 'position', 'opponent_team', 'player_name', 'week']]
    releventPositions_df = smallFantasy_df[smallFantasy_df['position'].isin(['QB', 'RB', 'WR', 'TE'])]
    
    # using top performers b/c thats usually more indicative of sos in fantasy football 
    topPerformers_df = releventPositions_df.sort_values(by='fantasy_points_ppr', ascending=False).groupby(['opponent_team', 'position']).head(10) # can use .head() if i want to see top performer averages
    
    
    # groupy by opponent team and position in order to find out how many total fantasy points each position has scored against them 
    defensePointsAgainst = topPerformers_df.groupby(['opponent_team', 'position'])['fantasy_points_ppr'].mean().reset_index()
    
    # assign a rank to each team based on the total fantasy points scored against them
    defensePointsAgainst['rank'] = defensePointsAgainst.groupby('position')['fantasy_points_ppr'].rank(ascending=True)
    
    defensePointsAgainst = defensePointsAgainst.sort_values(by=['position', 'rank'])
    
    # pivot the table so that the rankings are in the same row as the team name
    defenseRankings = defensePointsAgainst.pivot(index = "opponent_team", columns = "position", values = "rank")
    defensePPRPoints = defensePointsAgainst.pivot(index = "opponent_team", columns = "position", values = "fantasy_points_ppr")
    
    defenseRankings = defenseRankings.reset_index()
    defensePPRPoints = defensePPRPoints.reset_index()
    
    # merge the rankings and the average ppr points scored against each team
    defenseRankings = defenseRankings.merge(defensePPRPoints, on = "opponent_team", suffixes = ("_rank", "_ppr_avg"))

    return defenseRankings


def merge_defense_schedule (schedule_df, defense_df): 
    home_df = defense_df.add_suffix("_home")
    scheduleWithDefense = schedule_df.merge(home_df, left_on="home_team", right_on="opponent_team_home", how="left", suffixes=("", "_home"))
    scheduleWithDefense = scheduleWithDefense.merge(defense_df, left_on="away_team", right_on="opponent_team", how="left", suffixes=("", "_away"))
    scheduleWithDefense = scheduleWithDefense.drop(columns=["opponent_team_home", "opponent_team_away"], errors="ignore")
    return scheduleWithDefense

test = fantasy_defense_rankings(week_by_week2024)


merge_defense_schedule(schedule2024,test)

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,WR_ppr_avg_home,opponent_team,QB_rank,RB_rank,TE_rank,WR_rank,QB_ppr_avg,RB_ppr_avg,TE_ppr_avg,WR_ppr_avg
0,2024_01_BAL_KC,2024,REG,1,2024-09-05,Thursday,20:20,BAL,20.0,KC,...,15.870000,BAL,31.0,8.0,28.0,32.0,22.198000,14.520000,12.629999,28.207998
1,2024_01_GB_PHI,2024,REG,1,2024-09-06,Friday,20:15,GB,29.0,PHI,...,17.650000,GB,12.0,19.0,19.0,15.0,13.666667,16.780001,9.620000,17.960001
2,2024_01_PIT_ATL,2024,REG,1,2024-09-08,Sunday,13:00,PIT,18.0,ATL,...,20.260000,PIT,3.0,17.0,14.0,6.0,9.954000,16.660000,8.840000,15.660001
3,2024_01_ARI_BUF,2024,REG,1,2024-09-08,Sunday,13:00,ARI,28.0,BUF,...,16.430000,ARI,21.0,22.0,10.0,12.0,15.772000,17.160000,8.040000,16.820000
4,2024_01_TEN_CHI,2024,REG,1,2024-09-08,Sunday,13:00,TEN,17.0,CHI,...,14.370000,TEN,17.0,12.0,7.0,1.0,14.550000,15.802000,7.410000,13.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,2024_18_MIA_NYJ,2024,REG,18,2025-01-05,Sunday,13:00,MIA,NaN,NYJ,...,16.639999,MIA,2.0,21.0,6.0,2.0,9.554000,17.090000,7.170000,13.870000
268,2024_18_NYG_PHI,2024,REG,18,2025-01-05,Sunday,13:00,NYG,NaN,PHI,...,17.650000,NYG,25.0,23.0,3.0,13.0,17.210001,17.210001,6.220000,17.480000
269,2024_18_CIN_PIT,2024,REG,18,2025-01-05,Sunday,13:00,CIN,NaN,PIT,...,15.660001,CIN,29.0,15.0,31.0,22.5,19.184000,15.990001,13.110001,19.340000
270,2024_18_NO_TB,2024,REG,18,2025-01-05,Sunday,13:00,NO,NaN,TB,...,22.090000,NO,13.0,32.0,22.0,20.0,13.845999,21.990000,10.760000,19.130001


In [21]:
scheduleWithDefense[['away_team','home_team','QB_rank_home', 'RB_rank_home', 'TE_rank_home',
       'WR_rank_home', 'QB_ppr_avg_home', 'RB_ppr_avg_home', 'TE_ppr_avg_home',
       'WR_ppr_avg_home', 'opponent_team', 'QB_rank', 'RB_rank', 'TE_rank',
       'WR_rank', 'QB_ppr_avg', 'RB_ppr_avg', 'TE_ppr_avg', 'WR_ppr_avg']]

,away_team,home_team,QB_rank_home,RB_rank_home,TE_rank_home,WR_rank_home,QB_ppr_avg_home,RB_ppr_avg_home,TE_ppr_avg_home,WR_ppr_avg_home,opponent_team,QB_rank,RB_rank,TE_rank,WR_rank,QB_ppr_avg,RB_ppr_avg,TE_ppr_avg,WR_ppr_avg
0,BAL,KC,16.0,1.0,29.0,7.0,14.522000,10.000000,12.820000,15.870000,BAL,31.0,8.0,28.0,32.0,22.198000,14.520000,12.629999,28.207998
1,GB,PHI,4.0,2.0,5.0,14.0,11.968000,11.240000,6.700000,17.650000,GB,12.0,19.0,19.0,15.0,13.666667,16.780001,9.620000,17.960001
2,PIT,ATL,27.0,20.0,25.0,24.0,17.434000,16.830000,11.290000,20.260000,PIT,3.0,17.0,14.0,6.0,9.954000,16.660000,8.840000,15.660001
3,ARI,BUF,10.0,31.0,9.0,9.0,13.283999,21.439999,7.800000,16.430000,ARI,21.0,22.0,10.0,12.0,15.772000,17.160000,8.040000,16.820000
4,TEN,CHI,1.0,9.0,16.0,4.0,9.360000,15.150000,9.059999,14.370000,TEN,17.0,12.0,7.0,1.0,14.550000,15.802000,7.410000,13.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,MIA,NYJ,14.0,16.0,8.0,10.0,14.222000,16.280001,7.740000,16.639999,MIA,2.0,21.0,6.0,2.0,9.554000,17.090000,7.170000,13.870000
268,NYG,PHI,4.0,2.0,5.0,14.0,11.968000,11.240000,6.700000,17.650000,NYG,25.0,23.0,3.0,13.0,17.210001,17.210001,6.220000,17.480000
269,CIN,PIT,3.0,17.0,14.0,6.0,9.954000,16.660000,8.840000,15.660001,CIN,29.0,15.0,31.0,22.5,19.184000,15.990001,13.110001,19.340000
270,NO,TB,32.0,26.0,32.0,31.0,23.081999,18.150000,13.560000,22.090000,NO,13.0,32.0,22.0,20.0,13.845999,21.990000,10.760000,19.130001


In [ ]:
# creating a function to calculate the win/loss ratio for each team given the seasonal data
def win_loss (schedule_df): 
    # making the dataframe smaller so its easier to handle the data
    smallSchedule2024 = schedule_df[['result', 'away_team', 'away_score', 'home_team', 'home_score', 'week']]   
    previousGames = smallSchedule2024[schedule_df['result'].notnull()]
    
    
    previousGames['home_win'] = np.where(previousGames['result'] > 0, 1, 0)
    previousGames['away_win'] = np.where(previousGames['result'] < 0, 1, 0)
    
    ## not sure if all this is necessary. I think the best way to do it would be to just count the 1s and 0s to figure out the record
    # obtaining the week list
    weekList = previousGames['week'].unique()
    
    #going week by week to calculate the records
    for week in weekList: 
        # getting the data for each week at a time (to tally up the win/loss at the specific time of playing)
        previousWeekData = previousGames[previousGames['week'] <= week]
        
        # calculating the wins and losses for each team
        homeWins = previousWeekData.groupby('home_team')['home_win'].sum()
        awayWins = previousWeekData.groupby('away_team')['away_win'].sum()
        homeLosses = previousWeekData.groupby('home_team')['home_win'].count() - previousWeekData.groupby('home_team')['home_win'].sum()
        awayLosses = previousWeekData.groupby('away_team')['away_win'].count() - previousWeekData.groupby('away_team')['away_win'].sum()
        
        teamWins = homeWins.add(awayWins, fill_value=0)
        teamLosses = homeLosses.add(awayLosses, fill_value=0)
        
        # creates a mask so we get the right week in the origianl df
        mask = schedule_df['week'] == week

        # adding the win/loss record to the schedule_df
        schedule_df.loc[mask, 'home_team_win_record'] = schedule_df['home_team'].map(teamWins)
        schedule_df.loc[mask, 'home_team_loss_record'] = schedule_df['home_team'].map(teamLosses)
        schedule_df.loc[mask, 'away_team_win_record'] = schedule_df['away_team'].map(teamWins)
        schedule_df.loc[mask, 'away_team_loss_record'] = schedule_df['away_team'].map(teamLosses)

    return schedule_df


scheduleWithRecords = win_loss(schedule2024) 
previousGames = scheduleWithRecords[scheduleWithRecords['result'].notnull()]
previousGames
